In [1]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split

In [91]:
#Loading the data with pandas
math = pd.read_csv("../Data/math.csv")
read = pd.read_csv("../Data/read.csv")
business = pd.read_csv("../Data/BUSSINES.csv")
geo = pd.read_csv("../Data/geo.csv", low_memory=False)
funding = pd.read_csv("../Data/FUNDING.csv")
income = pd.read_csv("../Data/income.csv")
teachers = pd.read_csv("../Data/Teacher Ratios.csv")
title1 = pd.read_csv("../Data/title1.csv")
#Set indexs
math = math.set_index("math_ncessch")
read = read.set_index("read_ncessch")
business = business.set_index("zip_code")
funding["ncesid"] = pd.to_numeric(funding["ncesid"], errors="coerce")
funding = funding.set_index("ncesid")
geo = geo.set_index("ncessch")
income = income.set_index("zipcode")
teachers["ncessch"] = teachers["ncessch"].astype(np.int64)
teachers = teachers.set_index("ncessch")
title1["ncessch"] = title1["ncessch"].astype(np.int64)
title1 = title1.set_index("ncessch")

In [47]:
list(set(title1['ntnl_school_lunch_program_status']))

['MISSING',
 'Yes, under Community Eligibility Option (CEO)',
 'Yes, under Provision 2',
 'Yes, participating without using any Provision or the CEO',
 'No',
 'Yes, under Provision 1']

In [92]:
#Fix title1 data
title1 = title1[title1['title_i_status'] != 'M']
title1 = title1[title1['title_i_status'] != '-9']
title1 = title1[title1['title_i_eligibility'] != 'Missing']
title1 = title1[title1['title_i_eligibility'] != '-9']
title1 = title1[title1['ntnl_school_lunch_program_status'] != 'MISSING']
title1 = title1[title1['ntnl_school_lunch_program_status'] != '-9']
title1 = title1[title1['school_wide_title_i_eligibility'] != 'Missing']
title1 = title1[title1['school_wide_title_i_eligibility'] != '-9']
title1['ntnl_school_lunch_program_status_Yes'] = title1['ntnl_school_lunch_program_status'].apply(lambda x: 'Yes' in x)
title1 = pd.concat([title1.drop('title_i_eligibility',1),pd.get_dummies(title1["title_i_eligibility"], prefix='title_1_eligbility_')], axis=1)
title1 = pd.concat([title1.drop('school_wide_title_i_eligibility',1),pd.get_dummies(title1["school_wide_title_i_eligibility"], prefix='school_wide_title_i_eligibility')], axis=1)
title1 = pd.concat([title1.drop('ntnl_school_lunch_program_status',1),pd.get_dummies(title1["ntnl_school_lunch_program_status"], prefix='ntnl_school_lunch_program_status')], axis=1)

#Fix teachers
teachers= teachers[teachers['num_full_time'] >0]

In [93]:
combined = math[['math_leaid','math_all_grades_numvalid','math_all_grades_pctprof_low','math_all_grades_pctprof_high']].join([read[['read_all_grades_numvalid','read_all_grades_pctprof_low','read_all_grades_pctprof_high']],geo[['zip','locale','lat','lon']]], how="inner")
combined = combined.join(business[['num_establishments',"num_paid_employees", "first_quarter_payroll", "annual_payroll"]], how="inner", on="zip")
combined = combined.join(funding.drop(["idcensus","name","conum","csa","cbsa","enroll"],1), how="inner", on="math_leaid")
combined = combined.join(income.drop(["statefips",'state'], 1), how="inner", on="zip")
combined = combined.join(teachers["num_full_time"], how="inner")
combined = combined.join(title1[["title_i_status",'ntnl_school_lunch_program_status_Yes', 'title_1_eligbility__No',
       'title_1_eligbility__Not Applicable', 'title_1_eligbility__Yes',
       'school_wide_title_i_eligibility_No',
       'school_wide_title_i_eligibility_Not Applicable',
       'school_wide_title_i_eligibility_Yes',
       'ntnl_school_lunch_program_status_No',
       'ntnl_school_lunch_program_status_Yes, participating without using any Provision or the CEO',
       'ntnl_school_lunch_program_status_Yes, under Community Eligibility Option (CEO)',
       'ntnl_school_lunch_program_status_Yes, under Provision 1',
       'ntnl_school_lunch_program_status_Yes, under Provision 2']], how="inner")
combined = combined.drop(['zip','num_paid_employees',"first_quarter_payroll", "annual_payroll", 'math_leaid'],1)
combined = combined[combined['math_all_grades_pctprof_low'] >=0]
combined = combined[combined['read_all_grades_pctprof_low'] >=0]
combined['math_full_time'] = combined['math_all_grades_numvalid']/combined['num_full_time']
combined['read_full_time'] = combined['read_all_grades_numvalid']/combined['num_full_time']
math_y = (combined['math_all_grades_pctprof_low']+combined['math_all_grades_pctprof_high'])/2
read_y = (combined['read_all_grades_pctprof_low']+combined['read_all_grades_pctprof_high'])/2
math_x = combined.drop(["math_all_grades_pctprof_low","math_all_grades_pctprof_high","read_all_grades_pctprof_low","read_all_grades_pctprof_high"],1)
read_x = combined.drop(["math_all_grades_pctprof_low","math_all_grades_pctprof_high","read_all_grades_pctprof_low","read_all_grades_pctprof_high"],1)

In [97]:
combined['math_full_time'].describe()

count    67337.000000
mean         9.420826
std          9.814720
min          0.025750
25%          5.675509
50%          8.092669
75%         12.083903
max       1188.888889
Name: math_full_time, dtype: float64

In [98]:
math_X_train_val, math_X_test, math_y_train_val, math_y_test = train_test_split(math_x, math_y, test_size = .2, train_size = .8, random_state =42 )
math_X_train, math_X_val, math_y_train, math_y_val = train_test_split(math_X_train_val, math_y_train_val, test_size = .3, train_size = .7, random_state =42 )

In [99]:
#depth = 0
#max_score = 0
#for i in range(1,20):
#    print("running with max depth ", i)
#    rfr = RandomForestRegressor(n_estimators=100, max_depth=i, random_state=42)
#    trained = rfr.fit(math_X_train, math_y_train)
#    score = rfr.score(math_X_val, math_y_val)
#    print("validation score", score)
#    if score > max_score:
#        depth = i
#        max_score = score

#Optimal at 19 max depth
rfr = RandomForestRegressor(n_estimators=100, max_depth=19, random_state=42)
trained = rfr.fit(math_X_train, math_y_train)
print(rfr.score(math_X_test, math_y_test))

0.6585876554529014


In [100]:
def calculateDelta(x_test, y_test,estimator):
    predictions = np.array(estimator.predict(x_test))
    return np.mean(np.absolute(predictions - y_test))
calculateDelta(math_X_test, math_y_test, rfr)

9.896264757111886

In [101]:
features = pd.DataFrame({'scores':rfr.feature_importances_, 'names':math_X_train.columns})
features.sort_values(by='scores', ascending=False)

,names,scores
108,school_wide_title_i_eligibility_Yes,1.352455e-01
0,math_all_grades_numvalid,7.001228e-02
3,lat,5.726139e-02
110,"ntnl_school_lunch_program_status_Yes, particip...",4.870279e-02
4,lon,4.815499e-02
47,pct_title1,4.095215e-02
100,num_full_time,2.753569e-02
115,read_full_time,2.750027e-02
114,math_full_time,2.719349e-02
1,read_all_grades_numvalid,2.613605e-02
